In [2]:
import torch
import json
import logging
from lstm_crf.dataset import Dataset
from lstm_crf.args import Config

logger = logging.getLogger(__name__)

In [2]:
config = Config()
config.data_dir = 'data/LaptopReview'
config.bert_model_name_or_path = 'distilbert-base-uncased'
config.debug_mode = True
config.get_meta()

In [3]:
dataset = Dataset()

In [4]:
dataset.prepare(config, partition='test')

/opt/homebrew/Caskroom/miniforge/base/envs/py39/lib/python3.9/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoin

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 100/100 [00:40<00:00,  2.48it/s]


In [1]:
from dataclasses import dataclass


In [47]:
@dataclass
class Hello:
    a: int = 0
    b: int = 0

    def __setitem__(self, k, v):
        self.__dict__.update(zip(k, v) if type(k) is tuple else [(k, v)])

In [48]:
def convert(instance_class, **kwargs):
    data_points = list()
    keys = tuple(kwargs.keys())

    for feature_point_list in zip(*tuple(kwargs.values())):
        print(feature_point_list)
        inst = instance_class()
        inst[keys] = feature_point_list
        data_points.append(inst)

    return data_points

In [49]:
a = [1,2,3,4]
b = [5,6,7,8]

xs = convert(Hello, a=a, b=b)

(1, 5)
(2, 6)
(3, 7)
(4, 8)


In [50]:
xs

[Hello(a=1, b=5), Hello(a=2, b=6), Hello(a=3, b=7), Hello(a=4, b=8)]

In [52]:
def instance_list_to_feature_lists(instance_list: list, feature_names = None):
    if not feature_names:
        feature_names = list(instance_list[0].__dict__.keys())

    features_lists = list()
    for name in feature_names:
        features_lists.append([getattr(inst, name) for inst in instance_list])

    return features_lists


In [54]:
a, b = instance_list_to_feature_lists(xs)

In [57]:
x = [1,2,3,4,5]

In [58]:
y = x[2,3]

TypeError: list indices must be integers or slices, not tuple

In [3]:
data_dir = './data/MSIE/test_split.txt'

In [11]:
with open(data_dir, 'r', encoding='utf-8') as f:
    d = json.load(f)


In [9]:
len(d)

14

In [10]:
d[0]

{'id': 40,
 'text': ['their',
  'ter-polymers',
  'containing',
  'perylene',
  'units',
  'were',
  'synthesized',
  'using',
  'Suzuki',
  'coupling',
  'polymerization.',
  'The',
  'perylene',
  'units',
  'were',
  'introduced',
  'to',
  'improve',
  'the',
  'charge',
  '-',
  'transport',
  'ability',
  'by',
  'enhancing',
  'the',
  'π-π',
  'interaction',
  'between',
  'polymer',
  'chains.',
  'The',
  'resulting',
  'polymers',
  'were',
  'characterized',
  'by',
  '1H',
  'NMR',
  ',',
  'elemental',
  'analysis',
  ',',
  'DSC',
  ',',
  'and',
  'TGA.',
  'The',
  'synthesized',
  'polymers',
  'were',
  'soluble',
  'in',
  'common',
  'organic',
  'solvents',
  ',',
  'and',
  'formed',
  'smooth',
  'and',
  'uniform',
  'spin',
  '-',
  'coated',
  'thin',
  'films.',
  'All',
  'of',
  'the',
  'polymers',
  'studied',
  'were',
  'found',
  'to',
  'exhibit',
  'good',
  'thermal',
  'stability',
  ',',
  'losing',
  'less',
  'than',
  '5%',
  'of',
  'their',


In [12]:
x = d[0]

In [13]:
type(x)

dict

In [14]:
x.keys()

dict_keys(['id', 'text', 'entities', 'relations'])

In [15]:
x['text']

['their',
 'ter-polymers',
 'containing',
 'perylene',
 'units',
 'were',
 'synthesized',
 'using',
 'Suzuki',
 'coupling',
 'polymerization.',
 'The',
 'perylene',
 'units',
 'were',
 'introduced',
 'to',
 'improve',
 'the',
 'charge',
 '-',
 'transport',
 'ability',
 'by',
 'enhancing',
 'the',
 'π-π',
 'interaction',
 'between',
 'polymer',
 'chains.',
 'The',
 'resulting',
 'polymers',
 'were',
 'characterized',
 'by',
 '1H',
 'NMR',
 ',',
 'elemental',
 'analysis',
 ',',
 'DSC',
 ',',
 'and',
 'TGA.',
 'The',
 'synthesized',
 'polymers',
 'were',
 'soluble',
 'in',
 'common',
 'organic',
 'solvents',
 ',',
 'and',
 'formed',
 'smooth',
 'and',
 'uniform',
 'spin',
 '-',
 'coated',
 'thin',
 'films.',
 'All',
 'of',
 'the',
 'polymers',
 'studied',
 'were',
 'found',
 'to',
 'exhibit',
 'good',
 'thermal',
 'stability',
 ',',
 'losing',
 'less',
 'than',
 '5%',
 'of',
 'their',
 'weight',
 'upon',
 'heating',
 'to',
 'approximately',
 '350',
 '°',
 'C.',
 'Perylene-containing',
 'p

In [16]:
len(x['text'])

2807

In [17]:
x['entities']

[{'id': 461850, 'label': 'CN', 'start_offset': 3, 'end_offset': 4},
 {'id': 461851, 'label': 'CN', 'start_offset': 12, 'end_offset': 13},
 {'id': 461852, 'label': 'PN', 'start_offset': 77, 'end_offset': 79},
 {'id': 461853, 'label': 'PV', 'start_offset': 91, 'end_offset': 94},
 {'id': 461854, 'label': 'CN', 'start_offset': 95, 'end_offset': 96},
 {'id': 461855, 'label': 'PN', 'start_offset': 98, 'end_offset': 102},
 {'id': 461856, 'label': 'CN', 'start_offset': 105, 'end_offset': 106},
 {'id': 461857, 'label': 'CN', 'start_offset': 107, 'end_offset': 108},
 {'id': 461858, 'label': 'CN', 'start_offset': 122, 'end_offset': 123},
 {'id': 480694, 'label': 'CN', 'start_offset': 133, 'end_offset': 147},
 {'id': 461865, 'label': 'CN', 'start_offset': 151, 'end_offset': 152},
 {'id': 461866, 'label': 'PN', 'start_offset': 155, 'end_offset': 159},
 {'id': 461867, 'label': 'PN', 'start_offset': 161, 'end_offset': 163},
 {'id': 461868, 'label': 'CN', 'start_offset': 166, 'end_offset': 167},
 {'id

In [20]:
x['text'][107]

'PFD2TBT'